# Pruebas del método de volumen finito
- Óscar Alvarado
- Oscar Esquivel
- Jose Porras

In [1]:
import sys
# Windows:
# Actualizar ...
# Ubuntu: 
sys.path.append("../pythonFVM_Oscar/FVM")
import FiniteVolumeMethod as fvm  # Usa numba y sólo está disponible para python 2.7 y 3.4-3.7


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [967]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
""" 
@author: José ft. Óscar
"""

import numpy as np
import plotly.graph_objects as go

class Mesh():
    """
    Class that defines the scalar mesh in which FVM will be apllied. This class contains methods for defining
    borders, getting volumes/nodes positions, visualization of mesh, etc. The term volumes refers to points, 
    more specifically, the term volumes is used for the center of the volumes but the border nodes are not
    included. The terms nodes acounts fot all the points (interior and border) of the mesh. For every node
    there is a tag; the tags is an string beginning with I,D,N or S indicating whether the node is interior,
    dirichlet, neumann or source, accordingly. The atributes that are lists or arrays that contains an element 
    for every node, like the 'tags' atribute, are sorted sweeping in the X, then Y an finally Z diretion;
    e.g. tags=[tag1,tag2,tag3,tag4,tag5,...] corresponds to nodes coordinates sorted like [(1,1,1),(2,1,1),(3,1,1),(1,2,1),(2,2,1),...]
    """
    
    def __init__(self, dim, volumes = None, lengths = None):
        """
        Constructor of the Mesh class. If the parameters 'volumes' and 'lengths' are given then 
        the instance atributes are defined assuming the mesh is uniform via the uniform_grid() method.
        When volumes or lenghts are leaved as None then the domain nodes must be defined 
        using setDmonio() just after making the instance of the class.

        dim: number of dimensions in the physical domain (int)
        volumes: number of volumes in each dimension (tuple of ints or int)
        lengths: lenght of domain in each dimension (tuple of ints or int)
        """

        self.volumes = (1,1,1)
        self.lengths = (0.01, 0.01, 0.01)

        #----default values for positions and separations of the grid-------------
        self.coords = [(0.005,) for _ in range(3)] # Coordenadas de los centros de los volúmenes
        self.dominios = [tuple([0.]+[self.coords[i][0]]+[self.lengths[i]]) for i in range(3)]
        self.deltas = [(0,) for _ in range(3)]
        # Hasta aquí tenemos un cubito

        self.__tags = {} # El etiquetado de todos los nodos sin fronteras
        self.__tags_fronteras = {} # El etiquetado de las fronteras

        self.__autoMesh = True      # Al parecer es una bandera para calcular la posición de los nodos atmte
        self.__emptyCoord = True    # Nos dice si las coordenadas del dominio (self.coords_dom) están guardadas
        self.__pressed = False      # No sé

        self.__intWallNodes = None  # No sé
        
        self.dim = dim
        #---if a parameter is not a tuple, that parameter is transformed into a tuple
        if isinstance(volumes, int):  self.volumes = (volumes, 1, 1)
        if isinstance(lengths, (int, float)):  self.lengths = (lengths, lengths/10, lengths/10)
        
        # Si los parámetros son tuplas (pero no necesariamente sería una tupla de 3), arreglamos eso:
        if isinstance(volumes, tuple):
            faltan = 3 - len(volumes)
            self.volumes = volumes + tuple([1 for i in range(faltan)])
        if isinstance(lengths, tuple): 
            faltan = 3 - len(lengths)
            self.lengths = lengths + tuple([lengths[0]/10 for i in range(faltan)])
        #---------------------------------------------------------------
        
        # if volumes and lengths are given, initialize values acording to dimension
        if (volumes and lengths):       
            self.uniform_grid()
            self.init_tags()
            self.init_tags_fronteras()
    
    
    def uniform_grid(self):
        l = np.array(self.lengths) # Para el manejo con numpy
        v = np.array(self.volumes)
        d = l/v # Separación entre todos los nodos de cada dimensión
        start = d/2 # La frontera "inicial" del arreglo
        stop = l-d/2 # La frontera "final" del arreglo
        self.coords = [tuple(np.linspace(strt, stp, vol)) for strt, stp, vol in list(zip(start, stop, v))] # Meshgrid posible
        dominios = [np.insert(arr,(0,len(arr)),[0, l[idx]]) for idx, arr in enumerate(self.coords)] # Coordenadas + fronteras
        # Separación entre los nodos (Aquí hay que ver cómo es cuando tenemos un grid de 2x1x1 ya cuando se haga el FVM
        self.deltas = [self.set_deltas(dom)  if len(self.set_deltas(dom)) != 0 else (dom[-1],) for dom in dominios]
        self.dominios = [tuple(dom) for dom in dominios]
        #self.faces = [tuple((np.array(coords[:-1]) + np.array(coords[1:]))/2) for coords in self.coords]
        self.faces = [(self.dominios[idx][0],) + tuple((np.array(coords[:-1]) + np.array(coords[1:]))/2) \
                          + (self.dominios[idx][-1],) for idx, coords in enumerate(self.coords)]
        self.get_deltas_faces()
        self.get_grids()
    
    
    def set_deltas(self, dominio):
        """
        Método para obtener la distancia que hay entre los nodos
        """
        return tuple((dominio[1:]-dominio[:-1])[1:-1])


    # Creo que esto no se usará, pero estuvo chida la deducción, lo dejo de todos modos xd
    #def totalDomNodes(self):
    #    d_1 = 6*self.volumes[0] + 1
    #    d_2 = self.volumes[1]*d_1 - self.volumes[0]*(self.volumes[1] - 1)
    #    d_3 = self.volumes[2]*d_2 - self.volumes[0]*self.volumes[1]*(self.volumes[2] - 1)
    #    return d_3
    
    def init_tags_fronteras(self):
        """
        Método para etiquetar las fronteras dependiendo de la dimensión, sólo se les da la propiedad de existir o no
        existir.
        """
        self.__tags_fronteras = {}
        X, Y, Z = [len(dom) for dom in self.dominios]
        for z in range(Z):
            for y in range(Y):
                for x in range(X):
                    t = b = n = s = "Off"
                    e = w = "ON"
                    if self.dim > 1: 
                        n = s = "ON"
                        if self.dim == 3: t = b = "ON"
                    # El siguiente cacho de código es para saber si nos encontramos con una frontera
                    if x==0 or y==0 or z==0 or x==(X-1) or y==(Y-1) or z==(Z-1):
                        var = None
                        if y != 0 and y != (Y - 1):
                            if z != 0 and z != (Z - 1):
                                if x == 0: var = "W"; value = w
                                elif x == (X - 1): var = "E"; value = e
                                else: continue
                            elif x != 0 and x != (X - 1):
                                if z == 0: var = "B"; value = b
                                elif z == (Z - 1): var = "T"; value = t
                                else: continue
                            else: continue
                            self.__tags_fronteras[f"{x}{y}{z}"] = {"frontera": {var: value},
                                                 "coord": [self.dominios[0][x], self.dominios[1][y], self.dominios[2][z]],
                                                                  "cond": {}} 
                        elif z != 0 and z != (Z - 1):
                            if x != 0 and x != (X - 1):
                                if y == 0: var = "S"; value = s
                                elif y == (Y - 1) : var = "N"; value = n
                                self.__tags_fronteras[f"{x}{y}{z}"] = {"frontera": {var: value},
                                                 "coord": [self.dominios[0][x], self.dominios[1][y], self.dominios[2][z]],
                                                                      "cond": {}} 
                        else: continue
    
    def init_tags(self):
        """
        Método que etiqueta las caras adyacentes de cada volumen dependiendo de la geometría. Pone un 0 (cero) cuando es una 
        frontera, una 'F' cuando es una cara interna y un 'Off' cuando no se está contando esa cara por las dimensiones del 
        problema. 
        """
        self.__tags = {}
        X, Y, Z = self.volumes
        for z in range(1,Z+1):
            for y in range(1,Y+1):
                for x in range(1,X+1):                   
                    t = b = n = s = "Off"
                    e = w = "F"
                    if x == 1: w = {}
                    elif x == X: e = {}
                    if self.dim > 1:
                        n = s = "F"
                        if y == 1: s = {}
                        elif y == Y: n = {}
                        if self.dim == 3:
                            t = b = "F"
                            if z == 1: b = {}
                            elif z == Z: t = {}

                    self.__tags[f"{x}{y}{z}"] = {"E": e, "W": w, "N": n, "S": s, "T": t, "B": b, 
                                             "coord": [self.dominios[0][x], self.dominios[1][y], self.dominios[2][z]]}
            
    def tag_wall(self, direction, tag, value):
        """
        Método para etiquetar fronteras dada la dirección, el tipo de condición de frontera y el valor.
        """
        for key in self.__tags.keys():
            if isinstance(self.__tags[key][direction], dict):
                self.__tags[key][direction][tag] = value
        for key in self.__tags_fronteras.keys():
            if self.__tags_fronteras[key]["frontera"].get(direction) == "ON":
                self.__tags_fronteras[key]["cond"][tag] = value

    def tag_wall_dirichlet(self, direction, value, coords=None):
        """
        Método para etiquetar fronteras con condición de Dirichlet dados ciertos valores.
        """
        if coords:
            for idx, key in enumerate(coords):
                if key in list(self.__tags.keys()):
                    self.__tags[key][direction[idx]]["D"] = value[idx]
                elif key in list(self.__tags_fronteras.keys()):
                    self.__tags_fronteras[key]["cond"]["D"] =  value[idx]
        else:
            if isinstance(direction, list):
                for idx, direct in enumerate(direction):
                    self.tag_wall(direct, "D", value[idx])
            else:
                self.tag_wall(direction, "D", value)
                    
    def tag_wall_neumann(self, direction, value, coords=None):
        """
        Método para etiquetar fronteras con condición de Neumann dados ciertos valores.
        """
        if coords:
            for idx, key in enumerate(coords):
                if key in list(self.__tags.keys()):
                    self.__tags[key][direction[idx]]["N"] = value[idx]
                elif key in list(self.__tags_fronteras.keys()):
                    self.__tags_fronteras[key]["cond"]["N"] = value[idx]
        else:
            if isinstance(direction, list):
                for idx, direct in enumerate(direction):
                    self.tag_wall(direct, "N", value[idx])
            else:
                self.tag_wall(direction, "N", value)
                    
    def tag_wall_source(self, direction, value, coords=None):
        """
        Método para etiquetar fronteras con condición de Neumann dados ciertos valores.
        """
        if coords:
            for idx, key in enumerate(coords):
                if key in list(self.__tags.keys()):
                    self.__tags[key][direction[idx]]["S"] = value[idx]
                elif key in list(self.__tags_fronteras.keys()):
                    self.__tags_fronteras[key]["cond"]["S"] = value[idx]
        else:
            if isinstance(direction, list):
                for idx, direct in enumerate(direction):
                    self.tag_wall(direct, "S", value[idx])
            else:
                self.tag_wall(direction, "S", value)
                
                
    def tag_wall_insulated(self, direction, coords=None):
        """
        Método para etiquetar fronteras con condición de aislamiento.
        """
        if coords:
            for idx, key in enumerate(coords):
                if key in list(self.__tags.keys()):
                    self.__tags[key][direction[idx]]["I"] = None
                elif key in list(self.__tags_fronteras.keys()):
                    self.__tags_fronteras[key]["cond"]["I"] = None
        else:
            if isinstance(direction, list):
                for idx, direct in enumerate(direction):
                    self.tag_wall(direct, "I", None)
            else:
                self.tag_wall(direction, "I", None)
                
    
    def set_dominio(self, dominio, faces=None):
        """
        Método para definir el dominio de estudio dadas unas coordenadas en forma de tupla.
        """
        
        self.__autoMesh = False # La posición de los nodos no se calcula en automático
        
        # Si 'dominio' no es tupla, transforma 'dominio' a la tupla unidimensional (dominio,)
        # Tendría que ser una tupla de tuplas/listas/arreglos para que sea válido.
        if not isinstance(dominio, (tuple, int, float)): # Creo que si es una lista o un arreglo, no funciona enteros o float
            tupla = (tuple(dominio), self.dominios[1], self.dominios[2])
            dominio = tupla
        # Asigna los atributos de la mesh correspondientes    
        self.dominios = [tuple(dominio[i]) for i in range(3)]
        self.coords = [tuple(dominio[i][1:-1]) for i in range(3)]
        self.lengths = tuple([dominio[i][-1] for i in range(3)])
        self.volumes = tuple([len(dominio[i][1:-1]) for i in range(3)])
        #self.deltas = [self.set_deltas(np.array(dominio[i])) for i in range(3)]
        self.deltas = [self.set_deltas(np.array(dom))  if len(self.set_deltas(np.array(dom))) != 0 else (dom[-1],) for dom in self.dominios]
        
        if faces: 
            # Si me está pasando una lista (o sea, es de una dimensión)
            if isinstance(faces[0], (int, float)):
                self.faces = tuple(dominio[0]) + tuple(faces) + tuple(dominio[-1])
            else: # Suponemos aquí que nos está pasando una lista de listas (o tupla de tuplas)
                for idx, face_1dim in enumerate(faces):
                    faces[idx] = list(dominio[idx][0]) + tuple(face_1dim) + tuple(dominio[idx][-1])
                self.faces = faces
        else: 
            self.faces = [(self.dominios[idx][0],) + tuple((np.array(coords[:-1]) + np.array(coords[1:]))/2) \
                          + (self.dominios[idx][-1],) for idx, coords in enumerate(self.coords)]
        self.get_deltas_faces()
        self.init_tags()
        self.init_tags_fronteras()
        self.get_grids()
    
    def get_grid_deltas_dominios(self, axis=0, orientation="e"):
        deltas_dominios = []
        self.grid_deltas_dominios = np.array([])
        dominios = [np.array(dom) for dom in self.dominios]
        coords = [np.array(coord) for coord in self.coords]
        for direction in range(3):
            if direction != axis:
                if len(dominios[direction]) == 3:
                    deltas_dominios.append(dominios[direction][1])
                else:
                    deltas_dominios.append(coords[direction])
            else:
                deltas_dominios.append(dominios[direction][1:] - dominios[direction][:-1])
        if orientation == "e" or orientation == "n" or orientation == "t":
            deltas_dominios[axis] = deltas_dominios[axis][1:]
        else:
            deltas_dominios[axis] = deltas_dominios[axis][:-1]
        self.grid_deltas_dominios = np.meshgrid(deltas_dominios[0], deltas_dominios[1], deltas_dominios[2], 
                                                indexing='ij')
        return self.grid_deltas_dominios
    
    
    def get_deltas_faces(self):
        self.deltas_faces = []
        faces = [np.array(caras) for caras in self.faces]
        dominio = [np.array(doms) for doms in self.dominios]
        for direction in range(3):
            self.deltas_faces.append(faces[direction][1:] - faces[direction][:-1])
            
    def get_grids(self):
        self.grid_deltas = np.array([])
        self.grid_faces = np.array([])
        self.grid_coords = np.array([])
        deltas = self.deltas
        coords = self.coords
        faces = self.faces
        self.grid_deltas = np.meshgrid(deltas[0], deltas[1], deltas[2], indexing='ij')
        self.grid_coords = np.meshgrid(coords[0], coords[1], coords[2], indexing='ij')
        self.grid_faces = np.meshgrid(faces[0], faces[1], faces[2], indexing='ij')
                
    
    def info(self):
        """
        Método para imprimir información relevante del mallado
        """
        print('=====================================')
        print('     MESH INFORMATION   ')
        print('=====================================')
        print("\nMesh type: Cartesian")
        print(f"Number of dimensions of mesh: {self.dim}")
        variables = "X Y Z".split()
        for idx in range(self.dim):
            var = variables[idx]
            print(f"\n ----- {var}-axis -----")
            print(f"Number of {var.lower()} volumes: {self.volumes[idx]}")
            print(f"Lenght {var.lower()} of problem domain: {self.lengths[idx]}")
            print(f"List of {var.lower()} positions of volumes: \n{self.coords[idx]}")
            print(f"List of {var.lower()} positions of domain nodes: \n{self.dominios[idx]}")
            
            
    def draw(self):
        """
        Método para graficar la malla. Este método se invoca hasta que se hayan inizializado todas las condiciones
        de frontera.
        """
        # Graficamos las fronteras, sean o no activas
        dic_colors = {"D": "darkturquoise", "N": "red", "S": "magenta", "Off": "white", "I": "gray"}
        condiciones = [list(self.__tags_fronteras[key]["cond"].keys())[0] if list(self.__tags_fronteras[key]["frontera"].values())[0] == "ON" else "Off" for key in list(self.__tags_fronteras.keys())]
        colores = [dic_colors[cond] for cond in condiciones]
        # Obtenemos las coordenadas de las fronteras y de los nodos internos.
        coordenadas = [] # Aquí se pondrán las coordenadas de las fornteras
        coord = [] # Aquí las coordendas de los nodos internos
        for i in range(3):
            coordenadas.append([self.__tags_fronteras[key]["coord"][i] for key in list(self.__tags_fronteras.keys())])
            coord.append([self.__tags[key]["coord"][i] for key in list(self.__tags.keys())])
        fig = go.Figure(data = go.Scatter3d(x = coordenadas[0], y = coordenadas[1], z = coordenadas[2],
                                              mode = 'markers', marker = dict(color = colores, symbol = "square", size = 2)))
        fig.add_trace(go.Scatter3d(x = coord[0], y = coord[1], z = coord[2],
                                              mode = 'markers', marker = dict(color = "blue", size = 5)))
        fig.show()


    def get_area(self, direction = 0):
        """
        Método que regresa las áreas del volumen en la dirección indicada
        """
        perpendicular = [i for i in range(3) if i != direction]
        num_fronteras = self.volumes[direction]
        arreglo = [np.array([]) for _ in range(3)]
        arreglo[direction] = np.ones(num_fronteras)
        for i in perpendicular:
            arreglo[i] = self.deltas_faces[i]
        areas_grid = np.meshgrid(arreglo[0], arreglo[1], arreglo[2], indexing='ij')
        
        return areas_grid[perpendicular[0]]*areas_grid[perpendicular[1]]

    
    def areas_x(self):
        """
        Returns a 3-dimensional numpy array containing the areas of the faces in
        the X direction. The numpy array contains an area for each volume. 
        """
        return self.get_area()

    def areas_y(self):
        """
        Returns a 3-dimensional numpy array containing the areas of the faces in
        the Y direction. The numpy array contains an area for each volume. 
        """
        return self.get_area(direction=1)

    def areas_z(self):
        """
        Returns a 3-dimensional numpy array containing the areas of the faces in
        the Z direction. The numpy array contains an area for each volume. 
        """  
        return self.get_area(direction=2)
    
    def get_mask_boundaries(self, direction):
        tags_fronteras = self.__tags_fronteras
        condicion = []
        dict_cond = {"I":0, "N":0, "D":1}
        for tag in tags_fronteras:
            if list(tags_fronteras[tag]["frontera"].keys())[0] == direction:
                cond = list(tags_fronteras[tag]["cond"].keys())[0]
                condicion.append(dict_cond[cond])
        return condicion

In [968]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  3 21:14:10 2019

@author: jose
"""

import numpy as np
#from numpy import maximum as maxi
#from Diffusion import Diffusion
#import scipy.sparse as sp

class Coefficients():
    """
    Class that defines the coefficients that every node needs to form it's
    discrete equation that represent the diferential equation. All the coefficients
    needed for a scalar quantity (also called variable or unkown) are contained
    in one object (instance of this class).
    """    

    def __init__(self, mesh):
        self._dim = mesh.dim
        self._mesh = mesh
        self._volumes = mesh.volumes
        self._Su = np.zeros(self._volumes)  ;  self._Sp = None
        self._aN = None ; self._aS = None ; self._aT = None ; self._aB = None
        self.init_coefs()
        
    def init_coefs(self):
        """
        Método que inicializa los coeficientes en un arreglo tridimensional
        """
        vols = self._volumes
        self._aP = np.zeros(vols)
        dim = self._dim
        self._aE = np.zeros(vols)
        self._aW = np.zeros(vols)
        self._Sp = np.zeros(vols)
        if dim > 1:
            self._aN = np.zeros(vols)
            self._aS = np.zeros(vols)
        if dim == 3:
            self._aT = np.zeros(vols)
            self._aB = np.zeros(vols)
        
    def set_diffusion(self, gamma):
        """This method makes an instance of the Diffusion class and uses it to
        update the aP,aE,aW, ... atributes.
        
        Gamma: The diffusion coeffcient of the transport equation, if the diffusion
               coefficient is constant then a float can be given but in other cases
               a function of the coordinates, denoted as Gamma=f(x,y,z) should be 
               used. (float or function)"""
        
        dim = self._dim
        malla = self._mesh
        diffusion = Diffusion(malla, gamma)
        east_diff, source_e = diffusion.east_diffusion()
        west_diff, source_w = diffusion.west_diffusion()
        self._aE -= east_diff
        self._aW -= west_diff
        self._Sp -= source_e + source_w
        self._aP +=  self._aE + self._aW
        if dim > 1:
            north_diff, source_n = diffusion.north_diffusion()
            south_diff, source_s = diffusion.south_diffusion()
            self._aN -= north_diff
            self._aS -= south_diff
            self._Sp -= source_n + source_s
            self._aP += self._aN + self._aS
        if dim == 3:
            top_diff, source_t = diffusion.top_diffussion()
            bottom_diff, source_b = diffusion.bottom_diffusion()
            self._aT -= top_diff
            self._aB -= bottom_diff
            self._Sp -= source_t + source_b
            self._aP += top_diff + bottom_diff
        self._aP += self._Sp
            
    def get_sU(self):
        return self._Su
    
    def get_sP(self):
        return self._Sp
    
    def get_N(self):
        return self._N
    
    def get_aP(self):
        return self._aP
    
    def get_aE(self):
        return self._aE
    
    def get_aW(self):
        return self._aW

    def get_aN(self):
        return self._aN

    def get_aS(self):
        return self._aS

    def get_aT(self):
        return self._aT

    def get_aB(self):
        return self._aB

In [981]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  3 21:37:18 2019

@author: jose
"""

import numpy as np
#from Coefficients import Coefficients

class Diffusion():
    """
    Class that has the methods for calculating the modifications of the aP, aE, aW, ...
    coefficients due to the Diffusion term in the transport equation.
    """
    
    def __init__(self, mesh, gamma):
        """ 
        Saves the mesh and a function as attributes of the object.  
        
        gamma: function that can be evaluated to obtain the value of the difussion
        coeficient, Gamma, in a particular position, that is Gamma = f(x,y,z). If 
        funcionGamma is an int or float it is assumed that the difussion coefficient 
        is constant. (float/int/function)
        """
        
        self._mesh = mesh
        self._gammaconstante = 0.
        self._gamma = None
        
        if isinstance(gamma, (int,float)):
            self._gammaconstante = gamma
            self._gamma = self.funcionConst
        # Se asume que es una función en este caso
        else:
            self._gamma = gamma
          
    def funcionConst(self, x, y, z):
        return self._gammaconstante
    
    def east_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aE coefficient 
        caused by the diffusion effect.
        """
        mesh = self._mesh
        faces_x = mesh.faces[0]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=0, orientation="e")
        δ_x = δ_d_x_grid
        gamma_e = self._gamma(faces_x[1:], y , z) # Usando todas las de la derecha (east)
        areas = mesh.areas_x()
        source = areas.copy()
        areas[-1,:,:] = 0
        source[:-1,:,:] = 0
        east_d = gamma_e * areas / δ_x
        sp = gamma_e * source / δ_x
        condicion = malla.get_mask_boundaries(direction="E")
        sp[-1:,:,:] = sp[-1:,:,:]*np.array(condicion).reshape(sp[-1:,:,:].shape)
        return east_d, sp

    def west_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aW coefficient
        caused by the diffusion effect.
        """        
        mesh = self._mesh
        faces_x = mesh.faces[0]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=0, orientation="w")
        δ_x = δ_d_x_grid
        gamma_w = self._gamma(faces_x[:-1], y , z) # Usando todas las de la izquierda (west)
        areas = mesh.areas_x()
        source = areas.copy()
        areas[0,:,:] = 0
        source[1:,:,:] = 0
        west_d = gamma_w * areas / δ_x
        sp = gamma_w * source / δ_x
        condicion = malla.get_mask_boundaries(direction="W")
        sp[:1,:,:] = sp[:1,:,:]*np.array(condicion).reshape(sp[:1,:,:].shape)
        return west_d, sp

    def north_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aN coefficient
        caused by the diffusion effect.
        """
        mesh = self._mesh
        faces_y = mesh.faces[1]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=1, orientation="n")
        δ_y = δ_d_y_grid
        gamma_n = self._gamma(x, faces_y[1:] , z) # Usando todas las de arriba (north)
        areas = mesh.areas_y()
        source = areas.copy()
        areas[:,-1,:] = 0
        source[:,:-1,:] = 0
        north_d = gamma_n * areas / δ_y
        sp = gamma_n * source / δ_y
        condicion = malla.get_mask_boundaries(direction="N")
        sp[:,-1:,:] = sp[:,-1:,:]*np.array(condicion).reshape(sp[:,-1:,:].shape)
        return north_d, sp

    def south_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aS coefficient
        caused by the diffusion effect.
        """
        mesh = self._mesh
        faces_y = mesh.faces[1]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=1, orientation="s")
        δ_y = δ_d_y_grid
        gamma_s = self._gamma(x, faces_y[:-1] , z) # Usando todas las de abajo (south)
        areas = mesh.areas_y()
        source = areas.copy()
        areas[:,0,:] = 0
        source[:,1:,:] = 0
        south_d = gamma_s * areas / δ_y
        sp = gamma_s * source / δ_y
        condicion = malla.get_mask_boundaries(direction="S")
        sp[:,:1,:] = sp[:,:1,:]*np.array(condicion).reshape(sp[:,:1,:].shape)
        return south_d, sp

    def top_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aT coefficient
        caused by the diffusion effect.
        """
        mesh = self._mesh
        faces_z = mesh.faces[2]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=2, orientation="t")
        δ_z = δ_d_z_grid
        gamma_t = self._gamma(x, y , faces_z[1:]) # Usando todas las superiores (top)
        areas = mesh.areas_z()
        source = areas.copy()
        areas[:,:,-1] = 0
        source[:,:,:-1] = 0
        top_d = gamma_t * areas / δ_z
        sp = gamma_t * source / δ_z
        condicion = malla.get_mask_boundaries(direction="T")
        sp[:,:,-1:] = sp[:,:,-1:]*np.array(condicion).reshape(sp[:,:,-1:].shape)
        return top_d, sp

    def bottom_diffusion(self):
        """
        Gives a 3D numpy array that should be used to correct the aB coefficient
        caused by the diffusion effect.
        """
        mesh = self._mesh
        faces_z = mesh.faces[2]
        x, y, z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2])
        δ_d_x_grid, δ_d_y_grid, δ_d_z_grid = mesh.get_grid_deltas_dominios(axis=2, orientation="b")
        δ_z = δ_d_z_grid
        gamma_b = self._gamma(x, y, faces_z[:-1]) # Usando todas las inferiores (bottom)
        areas = mesh.areas_z()
        source = areas.copy()
        areas[:,:,0] = 0
        source[:,:,1:] = 0
        bottom_d = gamma_b * areas / δ_z
        sp = gamma_b * source / δ_z
        condicion = malla.get_mask_boundaries(direction="B")
        sp[:,:,:1] = sp[:,:,:1]*np.array(condicion).reshape(sp[:,:,:1].shape)
        return bottom_d, sp

## Una dimensión

In [986]:
#------------- Initial data definition ----------------
lx = 0.5 # meters
TA = 100 # °C 
TB = 500 # °C 
k  = 1000 # W/m.K 
nx  = 5 # number of volumes
#-------------------------------------------------------------

In [987]:
# ------------Mesh definition and border conditions ----
malla = Mesh(1, volumes=nx,lengths=(lx,0.1,0.1))
malla.info()

     MESH INFORMATION   

Mesh type: Cartesian
Number of dimensions of mesh: 1

 ----- X-axis -----
Number of x volumes: 5
Lenght x of problem domain: 0.5
List of x positions of volumes: 
(0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45)
List of x positions of domain nodes: 
(0.0, 0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45, 0.5)


In [988]:
#-------------Definimos fronteras ----------
malla.tag_wall_dirichlet('W',TA)
malla.tag_wall_dirichlet('E',TB)

In [989]:
malla.draw()

In [990]:
#--- creamos el objeto coeficientes --------------------------
coef = Coefficients(malla)
coef.set_diffusion(k)
print("\naP:")
print(coef.get_aP())
print("\nSp:")
print(coef.get_sP())
print("\naE:")
print(coef.get_aE())
print("\naW:")
print(coef.get_aW())


aP:
[[[-300.]]

 [[-200.]]

 [[-200.]]

 [[-200.]]

 [[-300.]]]

Sp:
[[[-200.]]

 [[   0.]]

 [[   0.]]

 [[   0.]]

 [[-200.]]]

aE:
[[[-100.]]

 [[-100.]]

 [[-100.]]

 [[-100.]]

 [[   0.]]]

aW:
[[[   0.]]

 [[-100.]]

 [[-100.]]

 [[-100.]]

 [[-100.]]]


---

## Una dimensión, no uniformidad

In [991]:
#------------- Initial data definition ----------------
lx = 0.5 # meters
TA = 100 # °C 
TB = 500 # °C 
k  = 1000 # W/m.K 
nx  = 6 # number of volumes
#-------------------------------------------------------------

In [992]:
# ------------Creamos la malla de acuerdo a la geometría definida por el problema
malla = Mesh(1, volumes=nx, lengths=(lx,0.1,0.1))
malla.set_dominio(np.array([0., 0.05, 0.1, 0.3, 0.48, 0.5]))
malla.info() # Se imprime información

     MESH INFORMATION   

Mesh type: Cartesian
Number of dimensions of mesh: 1

 ----- X-axis -----
Number of x volumes: 4
Lenght x of problem domain: 0.5
List of x positions of volumes: 
(0.05, 0.1, 0.3, 0.48)
List of x positions of domain nodes: 
(0.0, 0.05, 0.1, 0.3, 0.48, 0.5)


In [993]:
#-------------Definimos fronteras ----------
malla.tag_wall_dirichlet('W',TA)
malla.tag_wall_dirichlet('E',TB)
malla.draw()

In [994]:
#--- creamos el objeto coeficientes --------------------------
coef = Coefficients(malla)
coef.set_diffusion(k)
print("\naP es:")
print(coef.get_aP())
print("\naE es:")
print(coef.get_aE())
print("\naW es:")
print(coef.get_aW())
print("\nSp:")
print(coef.get_sP())


aP es:
[[[-400.        ]]

 [[-250.        ]]

 [[-105.55555556]]

 [[-555.55555556]]]

aE es:
[[[-200.        ]]

 [[ -50.        ]]

 [[ -55.55555556]]

 [[   0.        ]]]

aW es:
[[[   0.        ]]

 [[-200.        ]]

 [[ -50.        ]]

 [[ -55.55555556]]]

Sp:
[[[-200.]]

 [[   0.]]

 [[   0.]]

 [[-500.]]]


---

## Dos dimensiones
El problema 7.2 del malaskera es el chido para ver esto, junto con el ejemplo de juguete

In [982]:
#-------------Definición de datos iniciales ----------------
lx = 0.3 # meters
ly = 0.4
lz = 0.01
Te = 0 # ºC
Fw = 500000 # W/m²
Tn = 100 # ºC
Ts = 0 # ºC
nvx  = 3 # Número de volúmenes
nvy = 4
k = 1000
#-------------------------------------------------------------

In [983]:
#--------malla T -------------------------
malla = Mesh(2, volumes = (nvx, nvy), lengths = (lx, ly, lz))
malla.tag_wall_neumann("W", Tn)
malla.tag_wall_dirichlet("N", Fw)
malla.tag_wall_insulated(["E", "S"])
malla.info()

     MESH INFORMATION   

Mesh type: Cartesian
Number of dimensions of mesh: 2

 ----- X-axis -----
Number of x volumes: 3
Lenght x of problem domain: 0.3
List of x positions of volumes: 
(0.049999999999999996, 0.15, 0.25)
List of x positions of domain nodes: 
(0.0, 0.049999999999999996, 0.15, 0.25, 0.3)

 ----- Y-axis -----
Number of y volumes: 4
Lenght y of problem domain: 0.4
List of y positions of volumes: 
(0.05, 0.15000000000000002, 0.25000000000000006, 0.35000000000000003)
List of y positions of domain nodes: 
(0.0, 0.05, 0.15000000000000002, 0.25000000000000006, 0.35000000000000003, 0.4)


In [984]:
malla.draw()

In [985]:
#--- creamos el objeto coeficientes --------------------------
coef = Coefficients(malla)
coef.set_diffusion(k)
print("\naP:")
print(coef.get_aP())
print("\naE:")
print(coef.get_aE())
print("\naW:")
print(coef.get_aW())
print("\naN:")
print(coef.get_aN())
print("\naS:")
print(coef.get_aS())
print("\nSp:")
print(coef.get_sP())


aP:
[[[-20.]
  [-30.]
  [-30.]
  [-40.]]

 [[-30.]
  [-40.]
  [-40.]
  [-50.]]

 [[-20.]
  [-30.]
  [-30.]
  [-40.]]]

aE:
[[[-10.]
  [-10.]
  [-10.]
  [-10.]]

 [[-10.]
  [-10.]
  [-10.]
  [-10.]]

 [[  0.]
  [  0.]
  [  0.]
  [  0.]]]

aW:
[[[  0.]
  [  0.]
  [  0.]
  [  0.]]

 [[-10.]
  [-10.]
  [-10.]
  [-10.]]

 [[-10.]
  [-10.]
  [-10.]
  [-10.]]]

aN:
[[[-10.]
  [-10.]
  [-10.]
  [  0.]]

 [[-10.]
  [-10.]
  [-10.]
  [  0.]]

 [[-10.]
  [-10.]
  [-10.]
  [  0.]]]

aS:
[[[  0.]
  [-10.]
  [-10.]
  [-10.]]

 [[  0.]
  [-10.]
  [-10.]
  [-10.]]

 [[  0.]
  [-10.]
  [-10.]
  [-10.]]]

Sp:
[[[  0.]
  [  0.]
  [  0.]
  [-20.]]

 [[  0.]
  [  0.]
  [  0.]
  [-20.]]

 [[  0.]
  [  0.]
  [  0.]
  [-20.]]]


---

## Tres dimensiones

In [995]:
#-------------Definición de datos iniciales ----------------
lx = 0.5 # meters
ly = 0.5
lz = 0.5
TA = 100 # °C 
TB = 500 # °C 
k  = 1000 # W/m.K
nx = 5 # Número de volúmenes (5 volumenes tienen 6 fronteras)
ny = 5
nz = 6
#-------------------------------------------------------------

In [997]:
# ------------Creamos la malla de acuerdo a la geometría definida por el problema
malla = Mesh(3, volumes = (nx, ny, nz), lengths = (lx, ly, lz))
#malla.setDominio(np.array([0., 0.05, 0.1, 0.3, 0.48, 0.5]))
malla.info() # Se imprime información

     MESH INFORMATION   

Mesh type: Cartesian
Number of dimensions of mesh: 3

 ----- X-axis -----
Number of x volumes: 5
Lenght x of problem domain: 0.5
List of x positions of volumes: 
(0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45)
List of x positions of domain nodes: 
(0.0, 0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45, 0.5)

 ----- Y-axis -----
Number of y volumes: 5
Lenght y of problem domain: 0.5
List of y positions of volumes: 
(0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45)
List of y positions of domain nodes: 
(0.0, 0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45, 0.5)

 ----- Z-axis -----
Number of z volumes: 6
Lenght z of problem domain: 0.5
List of z positions of volumes: 
(0.041666666666666664, 0.125, 0.20833333333333331, 0.2916666666666667, 0.375, 0.4583333333333333)
List of z positions of domain nodes: 
(0.0, 0.041666666666666664, 0.125, 0.20833333333333331, 0.2916666666666667, 0.375, 0.4583333333333333, 0.5)


In [998]:
#-------------Definimos fronteras ----------
malla.tag_wall_neumann("W", TA)
malla.tag_wall_source("N", TA)
malla.tag_wall_dirichlet(["E", "S", "T", "B"], [TB, TB, TB, TA])

In [999]:
malla.draw()